In [10]:
from sympy import Symbol, symbols, Matrix, transpose, Transpose, python, lambdify, Derivative, MutableDenseMatrix, cos, sin
from sympy.utilities.lambdify import lambdastr
from sympy import init_printing

In [11]:
x, y, sai, u, v, r = symbols('x y psi u v r')
u1, u2 = symbols('u1 u2')

# gs = Matrix([[xs+(vs/dpsis)*(sin(psis+dpsis*dts)-sin(psis))],
#              [ys+(vs/dpsis)*(-cos(psis+dpsis*dts)+cos(psis))],
#              [psis+dpsis*dts],
#              [vs],
#              [dpsis]])

state = Matrix([x, y, sai, u, v, r])
U = Matrix([u1, u2])

In [15]:
M = Matrix([
    [25.8, 0, 0],
    [0, 33.8, 6.2],
    [0, 6.2, 2.76]
])

Bt = Matrix([
    [1, 0],
    [0, 0],
    [0, 1]
])

Minv = M.inv()

def eta(u, v, sai):
    return Matrix([[u], [v], [sai]])

def nu(x, y, r):
    return Matrix([[x], [y], [r]])

def C(V):
    u = V[0]
    v = V[1]
    r = V[2]
    c13 = -33.8*v -(6.2+6.2)*r/2
    c23 = 25.8*u
    return Matrix([
        [0, 0, c13],
        [0, 0, c23],
        [-c13, -c23, 0]
    ])

def D(V):
    u = V[0]
    v = V[1]
    r = V[2]
    # return -Matrix([
    #     [-12 - 2.1 * abs(u), 0, 0],
    #     [0, -17 - 4.5 * abs(v) , -0.2],
    #     [0, -0.5, -0.5 - 0.1 * abs(r)]
    # ])
    # Abs causing issue in differentiability
    return -Matrix([
        [-12 - 2.1 * u, 0, 0],
        [0, -17 - 4.5 * v , -0.2],
        [0, -0.5, -0.5 - 0.1 * -r]
    ])

def R(sai):
    return Matrix([
        [cos(sai), -sin(sai), 0],
        [sin(sai), cos(sai), 0],
        [0, 0, 1]
    ])

def fx(V, sai):
    a = R(sai)*V
    b = -Minv*(C(V) + D(V))*V
    c = Matrix.vstack(a, b).reshape(6, 1) # TODO
    return c


def B():
    a = Matrix([
        [0, 0],
        [0, 0],
        [0, 0]
    ])
    b = Minv*Bt
    c = Matrix.vstack(a, b)
    return c

# X = [x, y, sai, u, v, r]
def dx(X, t = None, U = Matrix([[0], [0]])):
    V = Matrix(X[3:])
    sai = X[2]
    return fx(V, sai) + B()*U

fx = dx(state, U=U)
fx

Matrix([
[                                                                                                                                                                                  u*cos(psi) - v*sin(psi)],
[                                                                                                                                                                                  u*sin(psi) + v*cos(psi)],
[                                                                                                                                                                                                        r],
[                                                                                      r*(0.24031007751938*r + 1.31007751937984*v) + u*(-0.0813953488372093*u - 0.465116279069767) + 0.0387596899224806*u1],
[r*(-0.011303967327888*r - 1.2982788798133*u + 0.0464556592765461) + u*(0.700845974329055*r + 3.82074095682614*v) - 0.11303967327888*u2 + v*(-2.9164235705951*u - 0.2264439

In [16]:
fx.jacobian(state)

Matrix([
[0, 0, -u*sin(psi) - v*cos(psi),                                   cos(psi),                                                     -sin(psi),                                                               0],
[0, 0,  u*cos(psi) - v*sin(psi),                                   sin(psi),                                                      cos(psi),                                                               0],
[0, 0,                        0,                                          0,                                                             0,                                                               1],
[0, 0,                        0,   -0.162790697674419*u - 0.465116279069767,                                            1.31007751937984*r,                         0.48062015503876*r + 1.31007751937984*v],
[0, 0,                        0, -0.597432905484247*r + 0.904317386231039*v, 0.904317386231039*u - 0.452887981330222*v - 0.798935239206535, -0.022607934655776*r - 0.59

In [17]:
Jf = lambdify(state, fx.jacobian(state), 'numpy')
Jf(1, 2, 1, 2, 1, 1)
# print(python(fx.jacobian(state), 'numpy'))
print(lambdastr(state, fx.jacobian(state)))

lambda x,y,psi,u,v,r: (ImmutableDenseMatrix([[0, 0, -u*math.sin(psi) - v*math.cos(psi), math.cos(psi), -math.sin(psi), 0], [0, 0, u*math.cos(psi) - v*math.sin(psi), math.sin(psi), math.cos(psi), 0], [0, 0, 0, 0, 0, 1], [0, 0, 0, -0.162790697674419*u - 0.465116279069767, 1.31007751937984*r, 0.48062015503876*r + 1.31007751937984*v], [0, 0, 0, -0.597432905484247*r + 0.904317386231039*v, 0.904317386231039*u - 0.452887981330222*v - 0.798935239206535, -0.022607934655776*r - 0.597432905484247*u + 0.0464556592765461], [0, 0, 0, -0.904317386231039*r - 4.92998833138856*v, -4.92998833138856*u + 1.01735705950992*v + 1.61355017502917, 0.123249708284714*r - 0.904317386231039*u - 0.285516336056009]]))
